# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

# Access maps with API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read csv data 
city_weather_df = pd.read_csv("cities.csv")
# display output
city_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Tasiilaq,65.6145,-37.6368,24.80,58,75,20.71,GL,1619063644
1,Port Alfred,-33.5906,26.8910,69.17,89,100,11.45,ZA,1619063703
2,Biltine,14.5333,20.9167,80.87,11,67,9.93,TD,1619063705
3,Baykit,61.6700,96.3700,34.05,82,42,4.07,RU,1619063706
4,Yarmouth,41.7057,-70.2286,46.40,70,90,13.80,US,1619063706
...,...,...,...,...,...,...,...,...,...
552,Gusau,12.1628,6.6614,81.55,50,100,13.15,NG,1619064193
553,Matagami,49.7502,-77.6328,15.80,79,1,3.44,CA,1619064157
554,Harper,4.3750,-7.7169,79.72,85,32,5.21,LR,1619064194
555,Kuldīga,56.9740,21.9572,37.40,100,90,9.22,LV,1619064195


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps
gmaps.configure(api_key=g_key)

# Set the Lat and Lng as locations
locations = city_weather_df[["Lat","Lng"]]

# Setup Heatmap layer variable
humidity = city_weather_df["Humidity"]

In [4]:
# Plot gmaps
map_fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Add heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, point_radius=3,max_intensity=max(humidity))

# Display gmap with heat layer
map_fig.add_layer(heat_layer)
map_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down DataFrame to ideal weather condition
ideal_weather_df = city_weather_df.loc[(city_weather_df["Wind Speed"] < 10) & (city_weather_df["Cloudiness"]== 0) &
                  (city_weather_df["Max Temp"] >= 70) & (city_weather_df["Max Temp"] <= 80)].dropna()
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
61,Vila Velha,-20.3297,-40.2925,71.60,88,0,3.44,BR,1619063755
74,Fuerte Olimpo,-21.0415,-57.8738,72.66,43,0,5.26,PY,1619063765
154,Saint-Philippe,-21.3585,55.7679,77.00,78,0,9.22,RE,1619063831
169,Puerto Suárez,-18.9500,-57.8000,72.09,55,0,3.74,BO,1619063844
252,Saint-Pierre,-21.3393,55.4781,77.00,78,0,9.22,RE,1619063912
277,Chaman,30.9236,66.4512,72.61,17,0,7.72,PK,1619063933
283,Taft,31.7471,54.2088,71.60,20,0,2.30,IR,1619063939
374,La Santisima Trinidad,-14.8333,-64.9000,73.51,95,0,3.85,BO,1619064028
400,Atar,20.5169,-13.0499,74.59,20,0,8.81,MR,1619064054
436,Tessalit,20.1986,1.0114,77.18,11,0,8.93,ML,1619064086


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into variable named hotel_df
hotel_df = ideal_weather_df.loc[:,["City","Country","Lat","Lng"]]

# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
61,Vila Velha,BR,-20.3297,-40.2925,
74,Fuerte Olimpo,PY,-21.0415,-57.8738,
154,Saint-Philippe,RE,-21.3585,55.7679,
169,Puerto Suárez,BO,-18.9500,-57.8000,
252,Saint-Pierre,RE,-21.3393,55.4781,
277,Chaman,PK,30.9236,66.4512,
283,Taft,IR,31.7471,54.2088,
374,La Santisima Trinidad,BO,-14.8333,-64.9000,
400,Atar,MR,20.5169,-13.0499,
436,Tessalit,ML,20.1986,1.0114,


In [7]:
# Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
# Iterrate through hotel_df to set parameters and collect hotel data
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    # Save hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
        #delay the code by one second to avoid an error
        time.sleep(1)
    print("--------------------Ending Search")

Retrieving Results for Index 61: Vila Velha.
Closest hotel in Vila Velha is Hotel Senac Ilha do Boi.
--------------------Ending Search
Retrieving Results for Index 74: Fuerte Olimpo.
Closest hotel in Fuerte Olimpo is Hotel AA.
--------------------Ending Search
Retrieving Results for Index 154: Saint-Philippe.
Closest hotel in Saint-Philippe is Les Embruns Du Baril.
--------------------Ending Search
Retrieving Results for Index 169: Puerto Suárez.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 252: Saint-Pierre.
Closest hotel in Saint-Pierre is Hôtel le Saint-Pierre.
--------------------Ending Search
Retrieving Results for Index 277: Chaman.
Closest hotel in Chaman is Sakhy Hotal Wah Mosafir Khana.
--------------------Ending Search
Retrieving Results for Index 283: Taft.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 374: La Santisima Trinidad.
Closest hotel in

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
# Add markers and info_box_content to show when user clicks it
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
map_fig.add_layer(markers)

# Display figure
map_fig

Figure(layout=FigureLayout(height='420px'))